In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        img = np.dstack((band_1, band_2, band_3))
        images.append(std_img(img))

    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,EarlyStopping
def ConvBlock(model, layers, filters):
    '''Create [layers] layers consisting of zero padding, a convolution with [filters] 3x3 filters and batch normalization. Perform max pooling after the last layer.'''
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(filters, (3, 3), activation='relu'))
        model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()

    # Input image: 75x75x3
    model.add(Lambda(lambda x: x, input_shape=(75, 75, 3)))
    ConvBlock(model, 1, 32)
    # 37x37x32
    ConvBlock(model, 1, 64)
    # 18x18x64
    ConvBlock(model, 1, 128)
    # 9x9x128
    ConvBlock(model, 1, 128)
    # 4x4x128
    model.add(Flatten())
    model.add(Dense(1,activation='sigmoid'))
    
    return model
print('model model')


model model


In [5]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch < 15:
        return 0.001
    elif epoch < 30:
        return 0.0005
    elif epoch < 45:
        return 0.0001
    else:
        return 0.00005

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.3,
            zoom_range=0.3,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20, verbose = 0, mode= 'min')
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s,early_stopping]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.53912, saving model to best_m.h5
 - 3s - loss: 0.6666 - acc: 0.7031 - val_loss: 0.5391 - val_acc: 0.7357
Epoch 2/100
Epoch 00002: val_loss improved from 0.53912 to 0.41526, saving model to best_m.h5
 - 2s - loss: 0.4107 - acc: 0.8139 - val_loss: 0.4153 - val_acc: 0.8204
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 2s - loss: 0.4139 - acc: 0.8141 - val_loss: 1.3074 - val_acc: 0.5885
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 2s - loss: 0.3474 - acc: 0.8408 - val_loss: 0.5607 - val_acc: 0.7706
Epoch 5/100
Epoch 00005: val_loss improved from 0.41526 to 0.34795, saving model to best_m.h5
 - 2s - loss: 0.3979 - acc: 0.8264 - val_loss: 0.3480 - val_acc: 0.8653
Epoch 6/100
Epoch 00006: val_loss did not improve
 - 2s - loss: 0.3559 - acc: 0.8350 - val_loss: 0.4472 - val_acc: 0.7706
Epoch 7/100
Epoch 00007: val_loss improved from 0.34795 to 0.20966, saving model to best_m.h5
 - 2s - loss: 0.3573 - acc: 0.8516 - val_loss:

Epoch 25/100
Epoch 00025: val_loss did not improve
 - 2s - loss: 0.2148 - acc: 0.9183 - val_loss: 0.2864 - val_acc: 0.8778
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 2s - loss: 0.2635 - acc: 0.8881 - val_loss: 0.2481 - val_acc: 0.8903
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 2s - loss: 0.2499 - acc: 0.8833 - val_loss: 0.3254 - val_acc: 0.8603
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 2s - loss: 0.2352 - acc: 0.9081 - val_loss: 0.4042 - val_acc: 0.8454
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 2s - loss: 0.2349 - acc: 0.9000 - val_loss: 0.3305 - val_acc: 0.8653
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 2s - loss: 0.2091 - acc: 0.9058 - val_loss: 0.2614 - val_acc: 0.9027
Epoch 31/100
Epoch 00031: val_loss improved from 0.23968 to 0.23221, saving model to best_m.h5
 - 2s - loss: 0.2059 - acc: 0.9117 - val_loss: 0.2322 - val_acc: 0.9002
Epoch 32/100
Epoch 00032: val_loss improved from 0.23221 to 0.22395, saving model to best_m.h5


Epoch 5/100
Epoch 00005: val_loss improved from 0.31331 to 0.28397, saving model to best_m.h5
 - 2s - loss: 0.3790 - acc: 0.8206 - val_loss: 0.2840 - val_acc: 0.8728
Epoch 6/100
Epoch 00006: val_loss did not improve
 - 2s - loss: 0.3738 - acc: 0.8400 - val_loss: 0.6774 - val_acc: 0.7656
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 2s - loss: 0.3614 - acc: 0.8522 - val_loss: 0.3242 - val_acc: 0.8529
Epoch 8/100
Epoch 00008: val_loss did not improve
 - 2s - loss: 0.3449 - acc: 0.8464 - val_loss: 0.3062 - val_acc: 0.8703
Epoch 9/100
Epoch 00009: val_loss improved from 0.28397 to 0.27012, saving model to best_m.h5
 - 2s - loss: 0.3868 - acc: 0.8278 - val_loss: 0.2701 - val_acc: 0.8628
Epoch 10/100
Epoch 00010: val_loss did not improve
 - 2s - loss: 0.3619 - acc: 0.8366 - val_loss: 0.3941 - val_acc: 0.8180
Epoch 11/100
Epoch 00011: val_loss improved from 0.27012 to 0.25113, saving model to best_m.h5
 - 2s - loss: 0.3238 - acc: 0.8658 - val_loss: 0.2511 - val_acc: 0.8678
Epoch 12/100

Epoch 67/100
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.1901 - acc: 0.9183 - val_loss: 0.1889 - val_acc: 0.9202
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.1793 - acc: 0.9225 - val_loss: 0.2000 - val_acc: 0.9002
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.1756 - acc: 0.9200 - val_loss: 0.1932 - val_acc: 0.9227
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 2s - loss: 0.1861 - acc: 0.9183 - val_loss: 0.1885 - val_acc: 0.9227
Epoch 71/100
Epoch 00071: val_loss did not improve
 - 2s - loss: 0.1817 - acc: 0.9208 - val_loss: 0.1981 - val_acc: 0.9077
Epoch 72/100
Epoch 00072: val_loss did not improve
 - 2s - loss: 0.1872 - acc: 0.9250 - val_loss: 0.1927 - val_acc: 0.9277
Epoch 73/100
Epoch 00073: val_loss did not improve
 - 2s - loss: 0.1733 - acc: 0.9333 - val_loss: 0.1930 - val_acc: 0.9302
Epoch 74/100
Epoch 00074: val_loss did not improve
 - 2s - loss: 0.1699 - acc: 0.9258 - val_loss: 0.1987 - val_acc: 0.9202
Epoch 75/100
Epo

In [6]:
import pickle
with open('../features/cnn_4_aug1_feat_add_early.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_4_aug1_sub_add_early.csv', index=False)
# aug_1 0.216
# this one, small range and earlystop 0.214

# pre 0.214
# new 0.214

0.214801966427
         id  is_iceberg
0  5941774d    0.024993
1  4023181e    0.984950
2  b20200e4    0.257269
3  e7f018bb    0.996811
4  4371c8c3    0.589607
